In [1]:
import salvus.namespace as sn
from salvus.material._details import material as md
from material_ela_constants.Elastic_Material import * 






--> Server: 'https://l.mondaic.com/licensing_server', User: 'bristol.support', Group: 'UniversityOfBristol'.
--> Negotiating 1 license instance(s) for 'SalvusMesh' [license version 1.0.0] for 1 seconds ...
--> Success! [Total duration: 0.44 seconds]


In [2]:
matl = Austenite()
material_unoriented = sn.material.elastic.hexagonal.TensorComponents.from_params(**matl.VTI_parameters())
from salvus.material._details import material as md
# Define a default orientation for comparison
orientation_of_grains_default = sn.material.orientation.ClockwiseAngle.from_params(angle_in_degrees=0)

# Convert the unoriented material with default orientation to solver form for 2D simulations
matl_background = md.to_solver_form(material_unoriented.with_orientation(orientation_of_grains_default), ndim=2)


In [3]:
orientation_angle = sn.material.orientation.ClockwiseAngle.from_params(angle_in_degrees=np.rad2deg(np.pi/4))

matl_orientated = md.to_solver_form(material_unoriented.with_orientation(orientation_angle), ndim=2)
matl_orientated

Field,Type,Value (or range)
RHO,constant,8.1e+03
C11,constant,2.7e+11
C12,constant,1.1e+11
C13,constant,1.2e+10
C22,constant,2.7e+11
C23,constant,1.2e+10
C33,constant,4.8e+10


salvus.material.elastic.hexagonal._TensorComponents2D

In [4]:
matl.rotated_tensor(np.pi/4, dim=2)


array([[2.74675e+11, 1.09875e+11, 1.15250e+10],
       [1.09875e+11, 2.74675e+11, 1.15250e+10],
       [1.15250e+10, 1.15250e+10, 4.78750e+10]])

In [ ]:
matl.tensor()

In [ ]:
import numpy as np

# example: VTI stiffness in material/principal axes (6x6 Voigt ordering [11,22,33,23,13,12])
C = matl.tensor()
# fill C with your VTI values:
# C[0,0]=C11; C[0,1]=C12; C[0,2]=C13; C[1,1]=C11; C[2,2]=C33...
# C[3,3]=C44; C[4,4]=C44; C[5,5]=C66

theta = np.pi/4   # tilt angle in radians
c = np.cos(theta); s = np.sin(theta)

L = np.array([
    [c*c, 0,   s*s, 0,     2*c*s, 0],
    [0,   1,   0,   0,     0,     0],
    [s*s, 0,   c*c, 0,    -2*c*s, 0],
    [0,   0,   0,   c,     0,    -s],
    [-c*s,0,   c*s, 0,  c*c - s*s, 0],
    [0,   0,   0,   s,     0,     c]
], dtype=float)

C_rot = L.dot(C).dot(L.T)   # rotated stiffness in global axes

# Extract 2D xz plane (Voigt indices 0->xx, 2->zz, 4->xz)
idx = [0, 2, 4]
C2D = C_rot[np.ix_(idx, idx)]

# Extract effective constants (Voigt order [11,22,33,23,13,12])
C11 = C_rot[0,0]
C12 = C_rot[0,1]
C13 = C_rot[0,2]
C22 = C_rot[1,1]
C23 = C_rot[1,2]
C33 = C_rot[2,2]
C44 = C_rot[3,3]   # corresponds to shear yz
C55 = C_rot[4,4]   # corresponds to shear xz
C66 = C_rot[5,5]   # corresponds to shear xy

print("Rotated stiffness constants (Voigt notation):")
print(f"C11 = {C11:.4e}")
print(f"C12 = {C12:.4e}")
print(f"C13 = {C13:.4e}")
print(f"C22 = {C22:.4e}")
print(f"C23 = {C23:.4e}")
print(f"C33 = {C33:.4e}")
print(f"C44 = {C44:.4e}")
print(f"C55 = {C55:.4e}")
print(f"C66 = {C66:.4e}")

print("\nC2D (xx, zz, xz):\n", C2D)
